# Convolutional Neural Network

### Importing the libraries

In [19]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [20]:
tf.__version__

'2.5.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set (Image augmentation)

### Image augmentation artificially creates training images through different ways of processing or combination of multiple processing, such as random rotation, shifts, shear and flips, etc.
#### Apply some transformations only to training set for aviod overfitting.
#### Otherwise we will get higher accuracy for training test and very low accuracy for test set.

In [21]:
# rescale = 1./255 : Apply feature scaling for each pixal : 0 - 255 -> 0 - 1
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        shear_range = 0.2,
        zoom_range = 0.2,
        horizontal_flip = True)

# target_size: Either None (default to original size) or tuple of ints (img_height, img_width).
training_set = train_datagen.flow_from_directory(
        "dataset/training_set",
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [22]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        "dataset/test_set",
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [23]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [24]:
# filters = 32 : classical architecture with 32 feture detectors (filters or kernels)
# kernel_size = 3 : kernel size 3 by 3
# input_shape = [64, 64, 3] : only for first convolution layer 64 by 64 images with RGB channels
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,activation = "relu", input_shape = [64, 64, 3]))

### Step 2 - Pooling (Apply max-pooling)

In [25]:
# pool_size = 2 : pooling window size 2 by 2 (recommend for max pooling)
# strides = 2 : pooling window sliding step size 2 pixels
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,strides = 2))

### Adding a second convolutional layer

In [26]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3,activation = "relu"))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2,strides = 2))

### Step 3 - Flattening

In [27]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [28]:
# relu good for all layers except to the final layer
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))

### Step 5 - Output Layer

In [29]:
# softmax for not binary predictions 
cnn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [30]:
cnn.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics=["accuracy"])
# batch_size = 32, Found 8000 images
# 32 * 250 = 8000

### Training the CNN on the Training set and evaluating it on the Test set

In [31]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 231s 906ms/step - loss: 0.6691 - accuracy: 0.5844 - val_loss: 0.6274 - val_accuracy: 0.6295
Epoch 2/25
250/250 [==============================] - 91s 364ms/step - loss: 0.6077 - accuracy: 0.6708 - val_loss: 0.5561 - val_accuracy: 0.7145
Epoch 3/25
250/250 [==============================] - 90s 360ms/step - loss: 0.5623 - accuracy: 0.7084 - val_loss: 0.5397 - val_accuracy: 0.7360
Epoch 4/25
250/250 [==============================] - 86s 345ms/step - loss: 0.5364 - accuracy: 0.7226 - val_loss: 0.5160 - val_accuracy: 0.7505
Epoch 5/25
250/250 [==============================] - 85s 339ms/step - loss: 0.5127 - accuracy: 0.7424 - val_loss: 0.4836 - val_accuracy: 0.7700
Epoch 6/25
250/250 [==============================] - 88s 351ms/step - loss: 0.4936 - accuracy: 0.7606 - val_loss: 0.5116 - val_accuracy: 0.7615
Epoch 7/25
250/250 [==============================] - 88s 351ms/step - loss: 0.4776 - accuracy: 0.7699 - val_loss: 0.4731 - val_a

## Part 4 - Making a single prediction

In [42]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img("dataset/single_prediction/cat_or_dog_2.jpg", target_size = (64, 64))
# Converts a PIL Image instance to a Numpy array.
test_image = image.img_to_array(test_image)
# Add image to a batch by adding extra dimension (cnn only accept batch for predict)
# axis = 0 : add extra dimension as first dimension ( batch[ [img1], [img2], .. ] )
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)

# for identfy cat and dog classes ( 1 or 0 )
print(training_set.class_indices)
# result[0][0]
#        |  |_____ First Image
#        |________ First Batch
if result[0][0] == 1:
    prediction = "Dog"
else:
    prediction = "Cat"

{'cats': 0, 'dogs': 1}


In [43]:
print(prediction)

Cat
